In [162]:
%load_ext autoreload
%autoreload 2
from datascience import *
from prob140 import *

from prob140.pykov import *
import prob140.pykov as pykov
# We could also do import prob140.pykov as pk
# and call pk.Vector, etc

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [267]:
import pandas as pd    

def matrix_to_pandas(matrix):
    all_states =sorted(matrix.states())
    target_states = ['S: '+(str(label)) for label in all_states]
    data = {source: [0]*len(all_states) for source in all_states}
    for (source,target),probability in matrix.items():
        data[target][all_states.index(source)] = probability
    data = {'T: '+str(label): values for label,values in data.items()}
    return pd.DataFrame(data,index=target_states)
    
def matrix_to_table(matrix):
    t = Table().with_columns('Source',[],'Target',[],'Probability',[])
    rows = [(source,target,probability) for (source,target),probability in matrix.items()]
    return t.with_rows(rows)

def table_to_vector(table):
    assert table.num_columns == 2, "You must have 2 columns for this task"
    label_column = table.column(0)
    prob_column = table.column(1)
    return pykov.Vector({label:prob for label,prob in zip(label_column,prob_column)})

def vector_to_table(vector,valueLabel='Probability'):
    t = Table().with_columns('State',[],valueLabel,[])
    rows = sorted(vector.items(),key=lambda x:x[0])
    return t.with_rows(rows)

def pykov_connection(function):
    def internal(*args,**kwargs):
        new_args = [(table_to_vector(argument) if isinstance(argument,Table) else argument) for argument in args]
        kwargs = {key:(table_to_vector(value) if isinstance(value,Table) else value) for key,value in kwargs}
        output = function(*new_args,**kwargs)
        if isinstance(output,pykov.Vector):
            return vector_to_table(output)
        if isinstance(output,pykov.Matrix):
            return matrix_to_pandas(output)
        return output
    return internal

In [268]:
class MC2:
    def __init__(self,pykov_chain):
        self.chain = pykov_chain
    def _repr_html_(self):
        return matrix_to_pandas(self.chain)._repr_html_()
    
    @pykov_connection
    def move(self,state):
        return self.chain.move(state)
    
    @pykov_connection
    def distribution_after_time(self,initial_distribution,time):
        return self.chain.pow(initial_distribution,time)
    
    @pykov_connection
    def steady_state(self):
        return self.chain.steady()
    
    def mean_first_passage_time_to(self,target_state):
        return vector_to_table(self.chain.mfpt_to(target_state),'Mean Time')
    
    def random_walk(self,steps,start=None,stop=None):
        return self.chain.walk(steps,start,stop)
    
    @pykov_connection
    def probability_of_walk(self,walk):
        return np.e**(self.chain.walk_probability(walk))
    
    def is_accessible(self,i,j):
        return self.chain.is_accessible(i,j)
    
    def communicates(self,i,j):
        return self.chain.communicates(i,j)
    
    @pykov_connection
    def accessibility_matrix(self):
        return self.chain.accessibility_matrix()
    
    
def toMarkovChain(table):
    assert table.num_columns == 3, \
    'Must have columns: source,target,probability'
    assert all([round(probsum,6)==1 for probsum in table.group(0,collect=sum).column(2)]), \
        'Transition probabilities must sum to 1 for each source state'
    dict_of_values = {(row[0],row[1]):row[2] for row in table.rows}
    return MC2(pykov.Chain(dict_of_values))

In [243]:
def transitionFunction(x,y):
    x,y= ord(x),ord(y) # This is just hacky to use letters instead of numbers
    if x == y:
        return .5
    if (x + 1)%5 == y%5:
        return .5
    return 0
    
transMatrix = Table().values('Source',['A','B','C','D','E'],\
                             'Target',['A','B','C','D','E']).probability_function(transitionFunction)

C:\ProgramData\Anaconda3\lib\site-packages\datascience\tables.py:362: FutureWarning: column lists are deprecated; pass each as an argument
  "column lists are deprecated; pass each as an argument", FutureWarning)
C:\Users\Dibya Jyoti Ghosh\Documents\GitHub\Stat 140\prob140\prob140\single_variable.py:381: UserWarning: Probabilities sum to 5.0
  warnings.warn("Probabilities sum to {0}".format(sum(values)))


In [262]:
transMatrix

Source,Target,Probability
A,A,0.5
A,B,0.5
A,C,0
A,D,0
A,E,0
B,A,0
B,B,0.5
B,C,0.5
B,D,0
B,E,0


In [269]:
mc =toMarkovChain(transMatrix)
mc

,T: A,T: B,T: C,T: D,T: E
S: A,0.5,0.5,0.0,0.0,0.0
S: B,0.0,0.5,0.5,0.0,0.0
S: C,0.0,0.0,0.5,0.5,0.0
S: D,0.0,0.0,0.0,0.5,0.5
S: E,0.5,0.0,0.0,0.0,0.5


In [270]:
mc.mean_first_passage_time_to('A')

State,Mean Time
B,8
C,6
D,4
E,2


In [265]:
startVector = Table().values(['A','B','C','D','E']).probability([.1,.4,.2,.1,.2])

In [271]:
mc.distribution_after_time(startVector,2)

State,Probability
A,0.15
B,0.2
C,0.275
D,0.225
E,0.15


In [273]:
mc.accessibility_matrix() # everything can reach everything

,T: A,T: B,T: C,T: D,T: E
S: A,1,1,1,1,1
S: B,1,1,1,1,1
S: C,1,1,1,1,1
S: D,1,1,1,1,1
S: E,1,1,1,1,1


In [253]:
random_walk = mc.random_walk(10,start='A') #good for simulations
random_walk

['A', 'A', 'E', 'E', 'E', 'E', 'D', 'C', 'C', 'B', 'B']

In [274]:
mc.probability_of_walk(['A','A','A','B','C'])

0.06250000000000001